# Online Retail Data Analysis

## Introduction
This project focuses on analyzing online retail data to extract insights about customer behavior, product performance, and sales trends. We will leverage data analytics and machine learning techniques to explore the dataset and uncover key patterns.


In [ ]:
!source ~/Documents/projects/Retail_Analytics/.venv/bin/activate

In [ ]:
import sys
print(sys.executable)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import json
import glob
import os
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Setting to make numbers easier to read on display
pd.options.display.float_format = '{:20.2f}'.format  #info: Format floats to show 2 decimal places for better readability

# Show all columns on output
pd.set_option('display.max_columns', 999)  #info: Display all columns in DataFrame outputs for visibility


# Data Exploration

In [ ]:
# df = pd.read_csv("/home/jefferyp/Documents/projects/sales_2024-Q4.csv")
filepath = '../projects/sales/sales'
all_files = glob.glob(os.path.join(filepath, "*.csv"))

df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [ ]:
df = df.rename(columns={'Product variant SKU at time of sale':'SKU'})

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
# drop rows with missing values (or use df.fillna(value) to fill)
# df = df.dropna()  # caution: this drops all rows with any missing value
df = df.loc[df['Customer ID'] != 0]

In [ ]:
df.describe()

# Data Cleaning

In [ ]:
# check for missing values
missing_values = df.isnull().sum()
no_customerID = df.where(df['Customer ID']=='0').count()
print("Zero CustID:\n", no_customerID)
print("Missing values:\n", missing_values)

In [ ]:
# Description and Customer ID have huge number of null value

In [ ]:
# check for duplicates
duplicates = df.duplicated().sum()
print("Number of duplicate rows:", duplicates)

# remove duplicates
# df = df.drop_duplicates()

In [ ]:
# convert 'InvoiceDate' to datetime
df['Day'] = pd.to_datetime(df['Day'])

# Exploratory Analysis

In [ ]:
# descriptive statistics
print(df.describe(include='all'))  # include='all' shows stats for categorical columns too

# Distribution of Quantities

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df['Quantity ordered'], bins=30, kde=True)
plt.title('Quantity Distribution')
plt.xlabel('Quantity ordered')
plt.ylabel('Frequency')
plt.show()

# Total Sales Over Time:

In [ ]:
sales_over_time = df.groupby(df['Day'].dt.date)['Gross sales'].sum()
plt.figure(figsize=(12, 6))
sales_over_time.plot()
plt.title('Total Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.show()


# Sales By State 

In [ ]:
sales_by_country = df.groupby('Billing region')['Gross sales'].sum().sort_values(ascending=False)
plt.figure(figsize=(12, 6))
sales_by_country.head(10).plot(kind='bar', color='skyblue')
plt.title('Top 10 State by Sales')
plt.xlabel('State')
plt.ylabel('Total Sales')
plt.show()

# Sales by City

In [ ]:
sales_by_country = df.groupby('Billing city')['Gross sales'].sum().sort_values(ascending=False)
plt.figure(figsize=(12, 6))
sales_by_country.head(10).plot(kind='bar', color='skyblue')
plt.title('Top 10 City by Sales')
plt.xlabel('City')
plt.ylabel('Total Sales')
plt.show()

In [ ]:
#Identify Outliers: Use boxplots to identify outliers in numerical columns like Price and Quantity.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x=df['Quantity ordered'])
plt.title('Boxplot of Quantity')
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x=df['Gross sales'])
plt.title('Boxplot of Unit Price')
plt.show()


In [ ]:
df.describe(include='O')  #info: Provides descriptive statistics for object-type (categorical) columns like count, unique values, most frequent (top) value, and its frequency.

In [ ]:
df[df["Customer ID"].isna()].head(10)

In [ ]:
df[df["Quantity ordered"] < 0].head(10) #checking Negative Quantity

In [ ]:
df["Order ID"] = df["Order ID"].astype("str") # short info: Convert 'Invoice' column to string type for string operations.
df[df["Order ID"].str.match("^\\d{15}$") == False] ## short info: Filter rows with 'Invoice' values that are not exactly 6 digits long.

In [ ]:
#Replace all digits in 'Invoice' with an empty string and get unique non-numeric values.
df["Order ID"].str.replace("[0-9]", "", regex=True).unique() 

In [ ]:
#finding above funcky stockcode that do not follow pattern with some of do not have customer etc
#checking every non match if they imp or not

In [ ]:
# df[df["SKU"].str.contains("^DOT")]

# Data Cleaning 

In [ ]:
cleaned_df = df.copy()

In [ ]:
# Convert 'Invoice' to string type, create a mask for rows with exactly 6-digit invoices, and filter 'cleaned_df' based on this mask.


In [ ]:
import warnings  # short info: Import the warnings library to control warning messages.
warnings.filterwarnings("ignore")  # short info: Suppress all warning messages.

In [ ]:
cleaned_df.dropna(subset=["Customer ID"], inplace=True)

In [ ]:
cleaned_df.describe()

In [ ]:
# short info: Calculate the number of rows in 'cleaned_df' where the 'Price' column is equal to 0.

In [ ]:
len(cleaned_df[cleaned_df["Gross sales"] == 0])

In [ ]:
#dealing issue having 0 value

In [ ]:
cleaned_df["Gross sales"].min()

In [ ]:
len(cleaned_df)/len(df) # drop 23 % records for 

In [ ]:
cleaned_df = cleaned_df[cleaned_df["Gross sales"] > 0.0]

# Feature Engineering

In [ ]:
cleaned_df.info()

In [ ]:
cleaned_df["SalesLineTotal"] = cleaned_df["Quantity ordered"] * cleaned_df["Gross sales"]

cleaned_df

In [ ]:
aggregated_df = cleaned_df.groupby(by="Customer ID", as_index=False) \
    .agg(
        MonetaryValue=("SalesLineTotal", "sum"),
        Frequency=("Order ID", "nunique"),
        LastInvoiceDate=("Day", "max")
    )

aggregated_df.head(5)

In [ ]:
max_invoice_date = aggregated_df["LastInvoiceDate"].max()
aggregated_df["Recency"] = (max_invoice_date - aggregated_df["LastInvoiceDate"]).dt.days
aggregated_df.head(5)

# Finding Outlier

In [ ]:
# Set the figure size for the entire plot
plt.figure(figsize=(15, 5))  # short info: Create a figure with a width of 15 inches and a height of 5 inches.

# Subplot 1: Histogram of Monetary Value
plt.subplot(1, 3, 1)  # short info: Create a 1x3 grid of subplots, and select the 1st subplot.
plt.hist(aggregated_df['MonetaryValue'], bins=10, color='skyblue', edgecolor='black')  # short info: Create a histogram for 'MonetaryValue' with 10 bins.
plt.title('Monetary Value Distribution')  # short info: Set the title for the first subplot.
plt.xlabel('Monetary Value')  # short info: Label the x-axis.
plt.ylabel('Count')  # short info: Label the y-axis.

# Subplot 2: Histogram of Frequency
plt.subplot(1, 3, 2)  # short info: Select the 2nd subplot in the 1x3 grid.
plt.hist(aggregated_df['Frequency'], bins=10, color='lightgreen', edgecolor='black')  # short info: Create a histogram for 'Frequency' with 10 bins.
plt.title('Frequency Distribution')  # short info: Set the title for the second subplot.
plt.xlabel('Frequency')  # short info: Label the x-axis.
plt.ylabel('Count')  # short info: Label the y-axis.

# Subplot 3: Histogram of Recency
plt.subplot(1, 3, 3)  # short info: Select the 3rd subplot in the 1x3 grid.
plt.hist(aggregated_df['Recency'], bins=20, color='salmon', edgecolor='black')  # short info: Create a histogram for 'Recency' with 20 bins.
plt.title('Recency Distribution')  # short info: Set the title for the third subplot.
plt.xlabel('Recency')  # short info: Label the x-axis.
plt.ylabel('Count')  # short info: Label the y-axis.

plt.tight_layout()  # short info: Adjust subplot parameters to give specified padding for a cleaner layout.
plt.show()  # short info: Display the plots.


In [ ]:
# Set the figure size for the entire plot
plt.figure(figsize=(15, 5))  # short info: Create a figure with a width of 15 inches and a height of 5 inches.

# Subplot 1: Boxplot of Monetary Value
plt.subplot(1, 3, 1)  # short info: Create a 1x3 grid of subplots, and select the 1st subplot.
sns.boxplot(data=aggregated_df['MonetaryValue'], color='skyblue')  # short info: Create a boxplot for 'MonetaryValue'.
plt.title('Monetary Value Boxplot')  # short info: Set the title for the first subplot.
plt.xlabel('Monetary Value')  # short info: Label the x-axis.

# Subplot 2: Boxplot of Frequency
plt.subplot(1, 3, 2)  # short info: Select the 2nd subplot in the 1x3 grid.
sns.boxplot(data=aggregated_df['Frequency'], color='lightgreen')  # short info: Create a boxplot for 'Frequency'.
plt.title('Frequency Boxplot')  # short info: Set the title for the second subplot.
plt.xlabel('Frequency')  # short info: Label the x-axis.

# Subplot 3: Boxplot of Recency
plt.subplot(1, 3, 3)  # short info: Select the 3rd subplot in the 1x3 grid.
sns.boxplot(data=aggregated_df['Recency'], color='salmon')  # short info: Create a boxplot for 'Recency'.
plt.title('Recency Boxplot')  # short info: Set the title for the third subplot.
plt.xlabel('Recency')  # short info: Label the x-axis.

plt.tight_layout()  # short info: Adjust subplot parameters to give specified padding for a cleaner layout.
plt.show()  # short info: Display the plots.


# Monetay and Freq have high outlier

## Establishing IQR

In [ ]:
M_Q1 = aggregated_df["MonetaryValue"].quantile(0.25)
M_Q3 = aggregated_df["MonetaryValue"].quantile(0.75)
M_IQR = M_Q3 - M_Q1

monetary_outliers_df = aggregated_df[(aggregated_df["MonetaryValue"] > (M_Q3 + 1.5 * M_IQR)) | (aggregated_df["MonetaryValue"] < (M_Q1 - 1.5 * M_IQR))].copy()
#short conclusion: This code creates a new DataFrame, 'monetary_outliers_df', containing rows from 'aggregated_df'
#where 'MonetaryValue' values are considered outliers, defined as being greater than 1.5 times the interquartile range (IQR) 
#above the third quartile (Q3) or below 1.5 times the IQR below the first quartile (Q1).

monetary_outliers_df.describe()

In [ ]:
F_Q1 = aggregated_df['Frequency'].quantile(0.25)
F_Q3 = aggregated_df['Frequency'].quantile(0.75)
F_IQR = F_Q3 - F_Q1

frequency_outliers_df = aggregated_df[(aggregated_df['Frequency'] > (F_Q3 + 1.5 * F_IQR)) | (aggregated_df['Frequency'] < (F_Q1 - 1.5 * F_IQR))].copy()

frequency_outliers_df.describe()

In [ ]:
# This code creates a new DataFrame, 'non_outliers_df', by filtering out rows from 'aggregated_df' that are present in either 'monetary_outliers_df' or 'frequency_outliers_df'. 
#It then provides descriptive statistics for the resulting DataFrame, summarizing the central tendency, dispersion, and shape of the dataset's distribution for non-outlier values.

non_outliers_df = aggregated_df[(~aggregated_df.index.isin(monetary_outliers_df.index)) & (~aggregated_df.index.isin(frequency_outliers_df.index))]

non_outliers_df.describe()

# Replot boxplot using Non Outlier

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.boxplot(data=non_outliers_df['MonetaryValue'], color='skyblue')
plt.title('Monetary Value Boxplot')
plt.xlabel('Monetary Value')

plt.subplot(1, 3, 2)
sns.boxplot(data=non_outliers_df['Frequency'], color='lightgreen')
plt.title('Frequency Boxplot')
plt.xlabel('Frequency')

plt.subplot(1, 3, 3)
sns.boxplot(data=non_outliers_df['Recency'], color='salmon')
plt.title('Recency Boxplot')
plt.xlabel('Recency')

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(non_outliers_df["MonetaryValue"], non_outliers_df["Frequency"], non_outliers_df["Recency"])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3D Scatter Plot of Customer Data')

plt.show()

In [ ]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(non_outliers_df[["MonetaryValue", "Frequency", "Recency"]])

scaled_data

In [ ]:
scaled_data_df = pd.DataFrame(scaled_data, index=non_outliers_df.index, columns=("MonetaryValue", "Frequency", "Recency"))

scaled_data_df

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(scaled_data_df["MonetaryValue"], scaled_data_df["Frequency"], scaled_data_df["Recency"])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3D Scatter Plot of Customer Data')

plt.show()

In [ ]:
max_k = 12

inertia = []
silhoutte_scores = []
k_values = range(2, max_k + 1)

for k in k_values:

    kmeans = KMeans(n_clusters=k, random_state=42, max_iter=1000)

    cluster_labels = kmeans.fit_predict(scaled_data_df)

    sil_score = silhouette_score(scaled_data_df, cluster_labels)

    silhoutte_scores.append(sil_score)

    inertia.append(kmeans.inertia_)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(k_values, inertia, marker='o')
plt.title('KMeans Inertia for Different Values of k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(k_values)
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(k_values, silhoutte_scores, marker='o', color='orange')
plt.title('Silhouette Scores for Different Values of k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.xticks(k_values)
plt.grid(True)

plt.tight_layout()
plt.show()

# Silhouette Score

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42, max_iter=1000)

cluster_labels = kmeans.fit_predict(scaled_data_df)

cluster_labels

In [ ]:
non_outliers_df["Cluster"] = cluster_labels

non_outliers_df

In [ ]:
cluster_colors = {0: '#1f77b4',  # Blue
                  1: '#ff7f0e',  # Orange
                  2: '#2ca02c',  # Green
                  3: '#d62728'}  # Red

colors = non_outliers_df['Cluster'].map(cluster_colors)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(non_outliers_df['MonetaryValue'], 
                     non_outliers_df['Frequency'], 
                     non_outliers_df['Recency'], 
                     c=colors,  # Use mapped solid colors
                     marker='o')

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3D Scatter Plot of Customer Data by Cluster')

plt.show()

In [ ]:
plt.figure(figsize=(12, 18))

plt.subplot(3, 1, 1)
sns.violinplot(x=non_outliers_df['Cluster'], y=non_outliers_df['MonetaryValue'], palette=cluster_colors, hue=non_outliers_df["Cluster"])
sns.violinplot(y=non_outliers_df['MonetaryValue'], color='gray', linewidth=1.0)
plt.title('Monetary Value by Cluster')
plt.ylabel('Monetary Value')

plt.subplot(3, 1, 2)
sns.violinplot(x=non_outliers_df['Cluster'], y=non_outliers_df['Frequency'], palette=cluster_colors, hue=non_outliers_df["Cluster"])
sns.violinplot(y=non_outliers_df['Frequency'], color='gray', linewidth=1.0)
plt.title('Frequency by Cluster')
plt.ylabel('Frequency')


plt.subplot(3, 1, 3)
sns.violinplot(x=non_outliers_df['Cluster'], y=non_outliers_df['Recency'], palette=cluster_colors, hue=non_outliers_df["Cluster"])
sns.violinplot(y=non_outliers_df['Recency'], color='gray', linewidth=1.0)
plt.title('Recency by Cluster')
plt.ylabel('Recency')

plt.tight_layout()
plt.show()

### Customer Segmentation Clusters

#### Cluster 0 (Blue): "Retain"
- **Rationale**: This cluster represents high-value customers who purchase regularly, though not always very recently. The focus should be on retention efforts to maintain their loyalty and spending levels.
- **Action**: Implement loyalty programs, personalized offers, and regular engagement to ensure they remain active.

---

#### Cluster 1 (Orange): "Re-Engage"
- **Rationale**: This group includes lower-value, infrequent buyers who haven’t purchased recently. The focus should be on re-engagement to bring them back into active purchasing behavior.
- **Action**: Use targeted marketing campaigns, special discounts, or reminders to encourage them to return and purchase again.

---

#### Cluster 2 (Green): "Nurture"
- **Rationale**: This cluster represents the least active and lowest-value customers, but they have made recent purchases. These customers may be new or need nurturing to increase their engagement and spending.
- **Action**: Focus on building relationships, providing excellent customer service, and offering incentives to encourage more frequent purchases.

---

#### Cluster 3 (Red): "Reward"
- **Rationale**: This cluster includes high-value, very frequent buyers, many of whom are still actively purchasing. They are your most loyal customers, and rewarding their loyalty is key to maintaining their engagement.
- **Action**: Implement a robust loyalty program, provide exclusive offers, and recognize their loyalty to keep them engaged and satisfied.

---

### Summary of Cluster Names:
- **Cluster 0 (Blue)**: "Retain"
- **Cluster 1 (Orange)**: "Re-Engage"
- **Cluster 2 (Green)**: "Nurture"
- **Cluster 3 (Red)**: "Reward"


In [ ]:
overlap_indices = monetary_outliers_df.index.intersection(frequency_outliers_df.index)

monetary_only_outliers = monetary_outliers_df.drop(overlap_indices)
frequency_only_outliers = frequency_outliers_df.drop(overlap_indices)
monetary_and_frequency_outliers = monetary_outliers_df.loc[overlap_indices]

monetary_only_outliers["Cluster"] = -1
frequency_only_outliers["Cluster"] = -2
monetary_and_frequency_outliers["Cluster"] = -3

outlier_clusters_df = pd.concat([monetary_only_outliers, frequency_only_outliers, monetary_and_frequency_outliers])

outlier_clusters_df

In [ ]:
cluster_colors = {-1: '#9467bd',
                  -2: '#8c564b',
                  -3: '#e377c2'}

plt.figure(figsize=(12, 18))

plt.subplot(3, 1, 1)
sns.violinplot(x=outlier_clusters_df['Cluster'], y=outlier_clusters_df['MonetaryValue'], palette=cluster_colors, hue=outlier_clusters_df["Cluster"])
sns.violinplot(y=outlier_clusters_df['MonetaryValue'], color='gray', linewidth=1.0)
plt.title('Monetary Value by Cluster')
plt.ylabel('Monetary Value')

plt.subplot(3, 1, 2)
sns.violinplot(x=outlier_clusters_df['Cluster'], y=outlier_clusters_df['Frequency'], palette=cluster_colors, hue=outlier_clusters_df["Cluster"])
sns.violinplot(y=outlier_clusters_df['Frequency'], color='gray', linewidth=1.0)
plt.title('Frequency by Cluster')
plt.ylabel('Frequency')

plt.subplot(3, 1, 3)
sns.violinplot(x=outlier_clusters_df['Cluster'], y=outlier_clusters_df['Recency'], palette=cluster_colors, hue=outlier_clusters_df["Cluster"])
sns.violinplot(y=outlier_clusters_df['Recency'], color='gray', linewidth=1.0)
plt.title('Recency by Cluster')
plt.ylabel('Recency')

plt.tight_layout()
plt.show()

### Customer Segmentation Outlier Clusters

#### Cluster -1 (Monetary Outliers): "PAMPER"
- **Characteristics**: High spenders but not necessarily frequent buyers. Their purchases are large but infrequent.
- **Potential Strategy**: Focus on maintaining their loyalty with personalized offers or luxury services that cater to their high spending capacity.

---

#### Cluster -2 (Frequency Outliers): "UPSELL"
- **Characteristics**: Frequent buyers who spend less per purchase. These customers are consistently engaged but might benefit from upselling opportunities.
- **Potential Strategy**: Implement loyalty programs or bundle deals to encourage higher spending per visit, given their frequent engagement.

---

#### Cluster -3 (Monetary & Frequency Outliers): "DELIGHT"
- **Characteristics**: The most valuable outliers, with extreme spending and frequent purchases. They are likely your top-tier customers who require special attention.
- **Potential Strategy**: Develop VIP programs or exclusive offers to maintain their loyalty and encourage continued engagement.


In [ ]:
cluster_labels = {
    0: "RETAIN",
    1: "RE-ENGAGE",
    2: "NURTURE",
    3: "REWARD",
    -1: "PAMPER",
    -2: "UPSELL",
    -3: "DELIGHT"
}

In [ ]:
full_clustering_df = pd.concat([non_outliers_df, outlier_clusters_df])

full_clustering_df

In [ ]:
full_clustering_df["ClusterLabel"] = full_clustering_df["Cluster"].map(cluster_labels)

full_clustering_df

# Visualization

In [ ]:
cluster_counts = full_clustering_df['ClusterLabel'].value_counts()
full_clustering_df["MonetaryValue per 100 pounds"] = full_clustering_df["MonetaryValue"] / 100.00
feature_means = full_clustering_df.groupby('ClusterLabel')[['Recency', 'Frequency', 'MonetaryValue per 100 pounds']].mean()

fig, ax1 = plt.subplots(figsize=(12, 8))

sns.barplot(x=cluster_counts.index, y=cluster_counts.values, ax=ax1, palette='viridis', hue=cluster_counts.index)
ax1.set_ylabel('Number of Customers', color='b')
ax1.set_title('Cluster Distribution with Average Feature Values')

ax2 = ax1.twinx()

sns.lineplot(data=feature_means, ax=ax2, palette='Set2', marker='o')
ax2.set_ylabel('Average Value', color='g')

plt.show()